In [1]:
import folium
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd

In [2]:
neighborhoods = gpd.read_file("./datasets/neighborhoods.geojson")
neighborhoods.head()

,name,geometry
0,Historic Buena Vista,(POLYGON ((-86.79511056795417 36.1757596496334...
1,Charlotte Park,(POLYGON ((-86.87459668651866 36.1575770268129...
2,Hillwood,(POLYGON ((-86.87613708067906 36.1355409894979...
3,West Meade,"(POLYGON ((-86.9038380396094 36.1255414807897,..."
4,White Bridge,(POLYGON ((-86.86321427797685 36.1288622289404...


In [3]:
urban_polygon = neighborhoods.loc[neighborhoods.name == "Urban Residents"]

# Print the head of the urban_polygon
urban_polygon.head()

,name,geometry
41,Urban Residents,(POLYGON ((-86.78122053774267 36.1645653773768...


In [4]:
urban_polygon['center'] = urban_polygon.loc[41,'geometry'].centroid
urban_polygon.head()

/Users/thin.nandar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,name,geometry,center
41,Urban Residents,(POLYGON ((-86.78122053774267 36.1645653773768...,POINT (-86.77756457127047 36.16127820928791)


# Create a folium location from the urban centroid

In [5]:
# Create urban_center from the urban_polygon center
urban_center = urban_polygon.loc[41,"center"]

# Print urban_center
print(urban_center)

# Create array for folium called urban_location
urban_location = [urban_center.y, urban_center.x]

# Print urban_location
print(urban_location)

POINT (-86.77756457127047 36.16127820928791)
[36.161278209287914, -86.77756457127047]


# Create a folium map of downtown Nashville

In [6]:
# Construct a folium map with urban_location
downtown_map = folium.Map(location = urban_location, zoom_start = 15)

# Display the map
display(downtown_map)

# Folium street map of the downtown neighborhood

In [7]:
# Create array for called folium_loc from the urban_polygon center point
point = urban_polygon.loc[41,"center"]
folium_loc = [point.y, point.x]

# Construct a map from folium_loc: downtown_map
downtown_map = folium.Map(location = folium_loc, zoom_start = 15)

# Draw our neighborhood: Urban Residents
folium.GeoJson(urban_polygon.geometry).add_to(downtown_map)

# Display the map
display(downtown_map)

# Adding markers for the public art

In [8]:
# From Chapter 2
art = pd.read_csv("./datasets/public_art.csv")

# Create a geometry column from lng & lat
art['geometry'] = art.apply(lambda x: Point(float(x.Longitude), float(x.Latitude)), axis=1)

# Create a GeoDataFrame from art and verify the type
art_geo = gpd.GeoDataFrame(art, crs = neighborhoods.crs, geometry = art.geometry)

neighborhood_art = gpd.sjoin(art_geo, neighborhoods, op = "within")
urban_art = neighborhood_art[neighborhood_art.name == "Urban Residents"]
urban_art.head()

,Title,Last Name,First Name,Location,Medium,Type,Description,Latitude,Longitude,Mapped Location,geometry,index_right,name
1,[Fourth and Commerce Sculpture],Walker,Lin,"333 Commerce Street, Nashville TN",NaN,Sculpture,NaN,36.16234,-86.77774,"(36.16234, -86.77774)",POINT (-86.77774000000001 36.16234),41,Urban Residents
4,A Story of Nashville,Ridley,Greg,"615 Church Street, Nashville TN",Hammered copper repousse,Frieze,"Inside the Grand Reading Room, this is a serie...",36.16215,-86.78205,"(36.16215, -86.78205)",POINT (-86.78205 36.16215),41,Urban Residents
21,Chet Atkins,Faxon,Russell,"Corner of Fifth Avenue North and Union Street,...",Bronze stool and guitar on a granite plynth,Sculpture,A sculpture of a young Chet Atkins seated on a...,36.16466,-86.78102,"(36.16466, -86.78102)",POINT (-86.78102 36.16466),41,Urban Residents
22,Children's Chairs For The Seasons,McGraw,Deloss,"615 Church Street, Nashville TN",Mixed Media - wood and paint,Furniture,chairs depicting the four seasons,36.16215,-86.78205,"(36.16215, -86.78205)",POINT (-86.78205 36.16215),41,Urban Residents
34,Foliated Scroll,Bloomer,Kent,"615 Church Street, Nashville TN",Aluminum,Frieze,a contemporary adaptation of a classical folia...,36.16215,-86.78205,"(36.16215, -86.78205)",POINT (-86.78205 36.16215),41,Urban Residents


In [9]:
# Iterate through the urban_art and print each part of tuple returned
for row in urban_art.iterrows():
    if row[0] == 1:
        print('first part: ', row[0])
        print('second part: ', row[1])

# Create a location and marker with each iteration for the downtown_map
for row in urban_art.iterrows():
    row_values = row[1] 
    location = [row_values['Latitude'], row_values['Longitude']]
    marker = folium.Marker(location = location)
    marker.add_to(downtown_map)

# Display the map
display(downtown_map)

first part:  1
second part:  Title                  [Fourth and Commerce Sculpture]
Last Name                                       Walker
First Name                                         Lin
Location             333 Commerce Street, Nashville TN
Medium                                             NaN
Type                                         Sculpture
Description                                        NaN
Latitude                                       36.1623
Longitude                                     -86.7777
Mapped Location                  (36.16234, -86.77774)
geometry           POINT (-86.77774000000001 36.16234)
index_right                                         41
name                                   Urban Residents
Name: 1, dtype: object


# Troubleshooting data issues

In [10]:
# Print the urban_art titles
print(urban_art.Title)

#Print the urban_art descriptions
print(urban_art.Description)

# Replace Nan and ' values in description
urban_art.Description.fillna('', inplace = True)
urban_art.Description = urban_art.Description.str.replace("'", "`")

#Print the urban_art descriptions again
print(urban_art.Description)

1                        [Fourth and Commerce Sculpture]
4                                   A Story of Nashville
21                                           Chet Atkins
22                     Children's Chairs For The Seasons
34                                       Foliated Scroll
39                                          Gone Fishing
40                             Happy Times at The Arcade
54                                     Johnny Cash Mural
55                                 La Storia della Terra
57                                 Les Paul Guitar Mural
59                                         Library Doors
60                               Lightning Wire Fountain
66     Nashville, A History through Maps and Aerial V...
67                                               Oh, Roy
68                                  Orpheus and Eurydice
75                               Piecing It All Together
104                                  The Birth of Apollo
105                            

/Users/thin.nandar/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Users/thin.nandar/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


# A map of downtown art

In [11]:
urban_location

[36.161278209287914, -86.77756457127047]

In [12]:
urban_polygon.geometry

41    (POLYGON ((-86.78122053774267 36.1645653773768...
Name: geometry, dtype: object

In [14]:
# Construct downtown map
downtown_map = folium.Map(location = urban_location, zoom_start = 15)
folium.GeoJson(urban_polygon.geometry).add_to(downtown_map)

# Create popups inside the loop you built to create the markers
for row in urban_art.iterrows():
    row_values = row[1]
    location = [row_values['Latitude'], row_values['Longitude']]
    popup = (str(row_values['Title']))
    #popup = (str(row_values['Title']) + ': ' + str(row_values['Description'])).replace("'", "`")
    #print(popup)
    marker = folium.Marker(location = location, popup = popup)
    #print(marker)
    #print()
    marker.add_to(downtown_map)

# Display the map.
display(downtown_map)